In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [26]:
class BasicBlock(nn.Module):
    
    def __init__(self, in_planes, planes, stride = 1):
        
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, 3, padding=1, bias=False, stride=stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride!=1 or in_planes!=planes :
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride = stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        
    
    def forward(self, x):
        
        identity = x
        
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += self.shortcut(identity)
        x = F.relu(x)
        
        return x


In [37]:
class ResNet(nn.Module):
    
    def __init__(self, block, filter_map, n ,num_classes=10):
        
        super().__init__()
        self.conv1 = nn.Conv2d(3, filter_map[0], kernel_size=3, bias=False)
        self.bn1 = nn.BatchNorm2d(filter_map[0])
        
        self.layer1 = self.makeResnetLayer(block, filter_map[0], n, stride=1)
        self.layer2 = self.makeResnetLayer(block, (filter_map[0], filter_map[1]), n, stride=2)
        self.layer3 = self.makeResnetLayer(block, (filter_map[1], filter_map[2]), n, stride=2)
        
        self.globalavgpool = nn.AdaptiveAvgPool2d(2)
        self.fc = nn.Linear(2*2*filter_map[2],num_classes)
    
    def makeResnetLayer(self, block, filters, n, stride):
        
        if stride!=1:
            in_planes, planes = filters
        else:
            in_planes, planes = filters, filters
            
        layer = []
        
        layer.append(block(in_planes, planes, stride))
        for i in range(n-1):
            layer.append(block(planes, planes))
        
        subBlock = nn.Sequential(*layer)
        return subBlock
    
    def forward(self, x):
        
        x = F.relu(self.bn1(self.conv1(x)))
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.globalavgpool(x)
        x = x.view(-1, self.find_shape(x))
        x = self.fc(x)
        x = F.softmax(x, dim =1)
        
        return x
    
    def find_shape(self, x):
        res = 1
        for dim in x[0].shape:
            res *= dim
        return res

In [38]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

In [39]:
batch_size = 128

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5, .5, .5), (.5, .5, .5))])

trainset = torchvision.datasets.CIFAR10(root = './data', download = True, train = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root = './data', download = True, train = False, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = True, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [40]:
len(testloader)

79

In [41]:
model = ResNet(BasicBlock, [16,32,64], 5)
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,50,80], gamma=0.3)

In [42]:
device = 'cpu'

In [43]:
model.train()

for epoch in range(85):
    
    running_loss = 0.0
    correct, total = 0, 0
    correct_epoch, total_epoch = 0, 0
    
    scheduler.step()
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        c = (torch.argmax(outputs,1) == labels)
        correct += torch.sum(c)
        total += 32
        correct_epoch += torch.sum(c)
        total_epoch += 32
        
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.7f accuracy: [%d, %d] (%d %%)' % (epoch + 1, i + 1, running_loss / 200, correct, total, 100*correct/total))
            running_loss = 0.0
            correct = 0
            total = 0
    
    print('<=========== accuracy after {} epoch is [{:d} {:d}] ({:d} %%) ===========>)'.format(epoch+1, correct_epoch, total_epoch, 100*correct_epoch/total_epoch))

print('Finished Training')


KeyboardInterrupt: 